<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Distributed-problem-solution" data-toc-modified-id="Distributed-problem-solution-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Distributed problem solution</a></span></li><li><span><a href="#nondistributed-solution" data-toc-modified-id="nondistributed-solution-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>nondistributed solution</a></span></li></ul></div>

In [ ]:
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt

# Description

We here study the possibility of carrying out implicit differentiation in a distributed manner. 

We first work on the very simple canonical example:

$$
\min f_1(x_1, y_1, \theta) + f_2(x_2, y_2, \theta)\\
\text{s.t. } y_1 = y_2
$$

We assume that 
$$
f_i = x_i^TA_i(\theta)x_i+y_i^TE_i(\theta)y_i+b_i(\theta)^Tx_i+c_i(\theta)^Ty_i+d_i(\theta)
$$

We denote by $z$ the set of variables, i.e. $z = (x_1, x_2, y_1, y_2)$, and the optimal solutions with a $\star$. 

In this short example, we will simply consider that we want to compute the Jacobian 
$$
\partial_\theta z^\star = - D_zG(z_\star)D_\theta G(z_\star),
$$
where $G$ denotes the gradients to the KKT conditions. 

In this context, we have that
$$
D_zG = \begin{bmatrix}
D_1 & 0 & B_1^T  \\
0 & D_2 & B_2^T\\
B_1 & B_2 & 0
\end{bmatrix}
$$

where $$D_i = \begin{bmatrix}
A_i & 0\\
0 & E_i
\end{bmatrix}
$$
and $$
B_1 = [0, 1], B_2 = [0, -1]. 
$$

In parallel
$$
D_\theta G = 
\begin{bmatrix}
D_\theta \nabla_z L\\
0
\end{bmatrix}
=
\begin{bmatrix}
D_\theta \nabla_{x_1} L\\
D_\theta \nabla_{x_2} L\\
...\\
0
\end{bmatrix}
=
\begin{bmatrix}
D_\theta A_1(\theta)x_1+b_1(\theta)\\
D_\theta A_2(\theta)x_1+b_2(\theta)\\
...\\
0
\end{bmatrix}
$$

We see that, due to the dual decomposition, the derivative with respect to parameters $\theta$ is also distributed. 

The question we want to ask is whether we can compute $\partial_\theta z^\star$ in a distributed manner. This would require inverting $D_zG$. Instead, we will try to achieve this with CG method. 

# Distributed problem solution

We first want to see whether we can simply solve a linear system in a distributed fashion, by repeated projections. In here, we will assume that x and y have dimension 2, and we therefore assimilate them in a variable $z_i$ which is dimension 4. 

We also will simply the local objectives so that they are
$$
z_i^TQz_i
$$

For now we assume that the vectors are shared, and known by everyone

What are the operations? 
- normalize
- multiply A and b
- dot product Ab and u
- multiply u
Therefore there should be four operations. 

We need to keep 3 vectors in memory
- x_t (current guess of the inversion)
- v_t (current latest basis vector)
- Atb (current multiplication of b)

In [ ]:
class Node():
    
    def __init__(self, number, n=2, theta_dim=1):
        Q = np.random.randn(2*n, 2*n)
        self.Q = Q@Q.T
        
        if number==1:
            self.B = np.concatenate([np.zeros((n, n)), np.eye(n)], axis = 1)
        elif number==2:
            self.B = np.concatenate([np.zeros((n, n)), -np.eye(n)], axis=1)
        
        
    def local_mult(self, z):
        
        return self.Q@z
    
    def coupling_mult(self, z):
        
        return self.B@z
        
    def norm(self, v1, v2):
        return v1.T@self.Q@v2
    

In [ ]:
class global_problem(): # a global problem which has a few nodes
    
    def __init__(self):
        self.n1 = Node(1)
        self.n2 = Node(2)
        
    def compute_norm(self, v1, v2):
        #vector v should have dimension (x1, y1, x2, y2, y1)
        v1_1, v2_1 = v1[:4], v2[:4]
        v1_2, v2_2 = v1[4:8], v2[4:8]
        v1_3, v2_3 = v1[8:], v2[8:]
        
        return (
        self.n1.norm(v1_1, v2_1) + self.n2.norm(v1_2, v2_2) + 
        self.n1.coupling_mult(v2_1)@v1_3 + self.n2.coupling_mult(v2_2)@v1_3 +
        self.n1.coupling_mult(v1_1)@v2_3 + self.n2.coupling_mult(v1_2)@v2_3
        )
    
    def normalize(self, v):
        return v/np.sqrt(self.compute_norm(v, v))
    
    def multiply(self, v):
        v1 = v[:4]
        v2 = v[4:8]
        v3 = v[8:]
        
        return np.concatenate(
            (self.n1.local_mult(v1)+self.n1.B.T@v3,
            self.n2.local_mult(v2) + self.n2.B.T@v3, 
            self.n1.B@v1 + self.n2.B@v2), axis=0
        )
    
    def construct_M(self):
        M = np.zeros((10, 10))
        M[:4, :4] = self.n1.Q
        M[4:8, 4:8] = self.n2.Q
        M[8:, :4] = self.n1.B
        M[:4, 8:] = self.n1.B.T
        M[8:, 4:8] = self.n2.B
        M[4:8, 8:] = self.n2.B.T
        return M
    
        
        
        

In [ ]:
prob = global_problem()

In [ ]:
M = prob.construct_M()

In [ ]:
b = np.random.randn(10)

In [ ]:
uk = prob.normalize(b)
xk = b
Akb = b
x = []
u = [uk]
for i in range(10):
    Akb = prob.multiply(Akb)
    vk = Akb - prob.compute_norm(Akb, uk)*uk
    uk = prob.normalize(vk)
    xk = xk + prob.compute_norm(b, uk)*uk
    x.append(xk)
    u.append(uk)

In [ ]:
prob.compute_norm(u[2], u[8])

In [ ]:
uk = prob.normalize(b)

In [ ]:
prob.compute_norm(M@M@b, uk)

# nondistributed solution

In [ ]:
M = np.random.randn(10, 10)

In [ ]:
# A = M@M.T

In [ ]:
A = M

In [ ]:
#CG method -- works
xk = np.zeros(A.shape[0])
r = b
rho = r.T@r
x = [xk]
rhos = [rho]
for k in range(10):
    if k == 0:
        p = r
    else:
        p = r + p*(rhos[-1]/rhos[-2])
    w = A@p
    alpha = rhos[-1]/(p@w)
    xk = xk + alpha*p
    r = r-alpha*w
    rhos.append(r.T@r)
    x.append(xk)

In [ ]:
#direct solution
x_opt = np.linalg.inv(A)@b

In [ ]:
x_opt

In [ ]:
xk

In [ ]:
diff = []
for x_ in x:
    diff.append((x_opt-x_).T@A@(x_opt-x_))
plt.plot(diff, 'o-')
plt.yscale('log')